In [32]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

from keras.layers import Dense, Dropout, Embedding, Flatten, Input, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from time import time
import datetime
from keras.models import Model
# from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from itertools import combinations
from scipy import sparse

import keras.backend as K

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [33]:
# data = train.append(test)
# data.reset_index(inplace=True)
# train_rows = train.shape[0]

# feature_results = []

# for target_g in ['car', 'ind', 'reg']:
#     features = [x for x in list(data) if target_g not in x]
#     target_list = [x for x in list(data) if target_g in x]
#     train_fea = np.array(data[features])
#     for target in target_list:
#         print(target)
#         train_label = data[target]
#         kfold = KFold(n_splits=5, random_state=218, shuffle=True)
#         kf = kfold.split(data)
#         cv_train = np.zeros(shape=(data.shape[0], 1))
#         for i, (train_fold, validate) in enumerate(kf):
#             X_train, X_validate, label_train, label_validate = \
#                 train_fea[train_fold, :], train_fea[validate, :], train_label[train_fold], train_label[validate]
#             dtrain = xgb.DMatrix(X_train, label_train)
#             dvalid = xgb.DMatrix(X_validate, label_validate)
#             watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
#             bst = xgb.train(params, dtrain, num_boost_round, evals=watchlist, verbose_eval=50,
#                             early_stopping_rounds=10)
#             cv_train[validate, 0] += bst.predict(xgb.DMatrix(X_validate), ntree_limit=bst.best_ntree_limit)
#         feature_results.append(cv_train)

# feature_results = np.hstack(feature_results)
# train_features = feature_results[:train_rows, :]
# test_features = feature_results[train_rows:, :]

In [34]:
def preprocessing_valid(texts, max_words, maxlen, n_train):
    
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    word_index = tokenizer.word_index
    data = pad_sequences(sequences, maxlen=maxlen)  
    indices = np.arange(data.shape[0])
    
    # shuffle
    np.random.seed(10101)
    np.random.shuffle(indices)
    data = data[indices]
    
    training_samples = n_train - 400000
    validation_samples = 300000
    holdout_samples = 100000

    X = {}
    X['train'] = data[:training_samples]
    X['valid'] = data[training_samples : training_samples + validation_samples]
    X['holdo'] = data[training_samples + validation_samples :]
    
    return X, word_index

def preprocessing_fulltrain(texts, max_words, maxlen, n_train):

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    word_index = tokenizer.word_index
    data = pad_sequences(sequences, maxlen=maxlen)  
    
    return data[:n_train], data[n_train:], word_index

In [35]:
with open('../input/text_features_lemm.pkl', 'rb') as f: 
    df_text = pickle.load(f)
texts = df_text.description.tolist()
print("text len:", len(texts))

n_train = 1503424
maxlen = 100
max_words = 10000

X_text, X_test_text = texts[:n_train], texts[:n_train]
X_text, word_index = preprocessing_valid(X_text, max_words, maxlen, n_train)
print('done')
# X_text['fulltrain'], X_text['test'], word_index_full = preprocessing_fulltrain(texts, max_words, maxlen, n_train)
# print('done')

# X_text['train'].shape, X_text['valid'].shape, X_text['fulltrain'].shape

text len: 2011862
done


In [36]:
embedding_index = {}
f = open('../input/wiki.ru.vec', encoding='utf-8')
for i, line in enumerate(f):
    if i == 0: continue
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs
    except ValueError:
        pass
f.close()
print("embedding index done, len:", len(embedding_index.keys()))


FileNotFoundError: [Errno 2] No such file or directory: '../input/wiki.ru.vec'

In [ ]:

embedding_dim = 300
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

NN

In [1]:
import pickle
with open('fea0.pkl', 'rb') as file: fea0 = pickle.load(file=file)
with open('labels.pkl', 'rb') as file: labels = pickle.load(file=file)
category_features = fea0['cat']
fea0.pop('cat');

In [3]:
tr_name = 'train'
te_name = 'valid'

for k in fea0.keys():
    fea0[k] = fea0[k].drop(['user_id'], axis=1)
category_features.remove('user_id')

In [4]:
train_cat = fea0[tr_name][category_features]
test_cat = fea0[te_name][category_features]

max_cat_values = []
for c in category_features:
    le = LabelEncoder()
    x = le.fit_transform(pd.concat([train_cat, test_cat])[c])
    train_cat[c] = le.transform(train_cat[c])
    test_cat[c] = le.transform(test_cat[c])
    max_cat_values.append(np.max(x))

C:\dev\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\dev\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [5]:
train_num = fea0[tr_name].drop(category_features, 1)
test_num = fea0[te_name].drop(category_features, 1)
train_num = train_num.replace([np.inf, -np.inf, np.nan], 0)
test_num = test_num.replace([np.inf, -np.inf, np.nan], 0)

scaler = StandardScaler()
scaler.fit(np.vstack([train_num, test_num]))
train_num = scaler.transform(train_num.values)
test_num = scaler.transform(test_num.values)

In [6]:
train_cat = train_cat.values
test_cat = test_cat.values

x_train_cat = []
for i in range(train_cat.shape[1]):
    x_train_cat.append(train_cat[:, i].reshape(-1, 1))
    
x_test_cat = []
for i in range(test_cat.shape[1]):
    x_test_cat.append(test_cat[:, i].reshape(-1, 1))
    
train_input = x_train_cat + [train_num]
test_input = x_test_cat + [test_num]

In [18]:

from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, CuDNNGRU, GlobalAveragePooling1D

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
       
def nn_model():
    
    inputs = []
    
    # text 
    input_text = Input(shape=(maxlen,))   
    x = Embedding(max_words, embedding_dim, input_length=maxlen, 
                  weights=[embedding_matrix], 
                  trainable=False
                 )(input_text)
    # convnet
    #x = CuDNNGRU(64, return_sequences=True)(x)
    x = Conv1D(64, 7, activation='relu')(x)
    x = MaxPooling1D(5)(x)
    x = Conv1D(64, 7, activation='relu')(x)
    x = MaxPooling1D(5)(x)
    x = Conv1D(64, 7, activation='relu')(x)
    x_max = GlobalMaxPooling1D()(x)
    x_avg = GlobalAveragePooling1D()(x)
    flatten_t = concatenate([x_max, x_avg])
    
    inputs.append(input_text)    
    
    # cat
    for e, c in enumerate(category_features):
        input_c = Input(shape=(1, ), dtype='int32')
        num_c = max_cat_values[e]
        embed_c = Embedding(
            num_c,
            16,
            input_length=1
        )(input_c)
        embed_c = Dropout(0.5)(embed_c)
        flatten_c = Flatten()(embed_c)
        inputs.append(input_c)

    # num
    input_num = Input(shape=(train_num.shape[1],), dtype='float32')
    inputs.append(input_num)
    
    # concatenate
    flatten = concatenate([flatten_c, input_num, flatten_t])
    
    # dense 
    fc1 = Dense(512, kernel_initializer='he_normal')(flatten)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.75)(fc1)

    fc1 = Dense(64, kernel_initializer='he_normal')(fc1)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.5)(fc1)

    outputs = Dense(1, kernel_initializer='he_normal', activation=None)(fc1)

    model = Model(inputs = inputs, outputs = [outputs])
    model.compile(loss=root_mean_squared_error, optimizer='adam')
    return (model)




In [16]:
model = nn_model()
model.fit(train_input, labels['train'], epochs=15, batch_size=512,
          validation_data=(test_input, labels['valid']))

Train on 1103424 samples, validate on 300000 samples
Epoch 1/15
1103424/1103424 [==============================] - 31s 28us/step - loss: 0.3118 - val_loss: 0.2335
Epoch 2/15
1103424/1103424 [==============================] - 30s 27us/step - loss: 0.2318 - val_loss: 0.2230
Epoch 3/15
1103424/1103424 [==============================] - 30s 27us/step - loss: 0.2277 - val_loss: 0.2226
Epoch 4/15
1103424/1103424 [==============================] - 30s 27us/step - loss: 0.2260 - val_loss: 0.2219
Epoch 5/15
1103424/1103424 [==============================] - 30s 27us/step - loss: 0.2250 - val_loss: 0.2227
Epoch 6/15
1103424/1103424 [==============================] - 30s 27us/step - loss: 0.2243 - val_loss: 0.2214
Epoch 7/15
1103424/1103424 [==============================] - 30s 27us/step - loss: 0.2240 - val_loss: 0.2213
Epoch 8/15
1103424/1103424 [==============================] - 30s 27us/step - loss: 0.2237 - val_loss: 0.2221
Epoch 9/15
1103424/1103424 [==============================] - 30s 2

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_67 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_64 (Embedding)        (None, 1, 16)        64          input_67[0][0]                   
__________________________________________________________________________________________________
dropout_70 (Dropout)            (None, 1, 16)        0           embedding_64[0][0]               
__________________________________________________________________________________________________
flatten_64 (Flatten)            (None, 16)           0           dropout_70[0][0]                 
__________________________________________________________________________________________________
input_68 (

In [ ]:
# num_seeds = 5
# begintime = time()
# if cv_only:
#     for s in xrange(num_seeds):
#         np.random.seed(s)
#         for (inTr, inTe) in kfold.split(X, train_label):
#             xtr = X[inTr]
#             ytr = train_label[inTr]
#             xte = X[inTe]
#             yte = train_label[inTe]

#             xtr_cat = X_cat[inTr]
#             xte_cat = X_cat[inTe]

#             # get xtr xte cat
#             xtr_cat_list, xte_cat_list = [], []
#             for i in xrange(xtr_cat.shape[1]):
#                 xtr_cat_list.append(xtr_cat[:, i].reshape(-1, 1))
#                 xte_cat_list.append(xte_cat[:, i].reshape(-1, 1))

#             xtr_cat_list.append(xtr)
#             xte_cat_list.append(xte)

#             model = nn_model()
#             def get_rank(x):
#                 return pd.Series(x).rank(pct=True).values
#             model.fit(xtr_cat_list, ytr, epochs=20, batch_size=512, verbose=2, validation_data=[xte_cat_list, yte])
#             cv_train[inTe] += get_rank(model.predict(x=xte_cat_list, batch_size=512, verbose=0)[:, 0])
#             print(Gini(train_label[inTe], cv_train[inTe]))
#             cv_pred += get_rank(model.predict(x=x_test_cat, batch_size=512, verbose=0)[:, 0])
#         print(s)
#         print(Gini(train_label, cv_train / (1. * (s + 1))))
#         print(str(datetime.timedelta(seconds=time() - begintime)))
#     if save_cv:
#         pd.DataFrame({'id': test_id, 'target': get_rank(cv_pred * 1./ (NFOLDS * num_seeds))}).to_csv('../model/keras5_pred.csv', index=False)
#         pd.DataFrame({'id': train_id, 'target': get_rank(cv_train * 1. / num_seeds)}).to_csv('../model/keras5_cv.csv', index=False)
        